In [1]:
import os
import os.path
import random
import gc
import numpy as np
import pandas as pd
import scipy.sparse
from tqdm import tqdm

In [2]:
import warnings 
warnings.filterwarnings('ignore')

In [3]:
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from IPython.core.pylabtools import figsize

In [4]:
sns.set()

In [5]:
import pickle

def dump_pickle(file, filename):
    outfile = open(filename, 'wb')
    pickle.dump(file, outfile)
    outfile.close()

def load_pickle(filename):
    infile = open(filename, 'rb')
    file = pickle.load(infile)
    infile.close()
    return file

In [6]:
DATA_DIR = '../input/open-problems-multimodal'
%ls $DATA_DIR -lh

total 27G
-rw-r--r-- 1 nobody nogroup 2.3G Sep  7 19:38 evaluation_ids.csv
-rw-r--r-- 1 nobody nogroup 9.4M Sep  7 19:37 metadata.csv
-rw-r--r-- 1 nobody nogroup 230K Sep  7 19:37 metadata_cite_day_2_donor_27678.csv
-rw-r--r-- 1 nobody nogroup 805M Sep  7 19:37 sample_submission.csv
-rw-r--r-- 1 nobody nogroup 1.6G Sep  7 19:38 test_cite_inputs.h5
-rw-r--r-- 1 nobody nogroup 294M Sep  7 19:37 test_cite_inputs_day_2_donor_27678.h5
-rw-r--r-- 1 nobody nogroup 6.1G Sep  7 19:39 test_multi_inputs.h5
-rw-r--r-- 1 nobody nogroup 2.4G Sep  7 19:38 train_cite_inputs.h5
-rw-r--r-- 1 nobody nogroup  37M Sep  7 19:37 train_cite_targets.h5
-rw-r--r-- 1 nobody nogroup  11G Sep  7 19:40 train_multi_inputs.h5
-rw-r--r-- 1 nobody nogroup 3.0G Sep  7 19:38 train_multi_targets.h5


## Read Data

In [7]:
%%time
train_inp = pd.read_hdf(f'{DATA_DIR}/train_cite_inputs.h5')
train_inp_cols = train_inp.columns

CPU times: user 28.1 s, sys: 5.29 s, total: 33.4 s
Wall time: 52.6 s


In [8]:
%%time
test_inp = pd.read_hdf(f'{DATA_DIR}/test_cite_inputs.h5')

CPU times: user 19 s, sys: 2.96 s, total: 21.9 s
Wall time: 35 s


In [9]:
%%time
train_tar = pd.read_hdf(f'{DATA_DIR}/train_cite_targets.h5')
train_tar_cols = train_tar.columns

CPU times: user 176 ms, sys: 39.7 ms, total: 216 ms
Wall time: 615 ms


## Data Preprocessing

Find columns with all zeroes

In [10]:
%%time
zero_cols = []
for idx, col in enumerate(train_inp_cols, 0):
    if idx % 1000 == 0:
        print(idx)
    if len(train_inp[col].unique()) == 1 or len(test_inp[col].unique()) == 1:
        zero_cols.append(col)
print('Number of columns with zero values only (Train or Test):', 
      len(zero_cols))

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
Number of columns with zero values only (Train or Test): 1194
CPU times: user 39.7 s, sys: 151 ms, total: 39.9 s
Wall time: 39.9 s


In [11]:
%%time
train_inp = train_inp.drop(zero_cols, axis=1)
train_inp_cols = train_inp.columns
test_inp = test_inp.drop(zero_cols, axis=1)
train_inp.shape, test_inp.shape

CPU times: user 1.24 s, sys: 1.96 s, total: 3.2 s
Wall time: 3.19 s


((70988, 20856), (48663, 20856))

In [12]:
# %%time
# np.min(train_inp.min().unique()), np.max(train_inp.max().unique())

In [13]:
from sklearn.preprocessing import StandardScaler, Normalizer

In [14]:
sc = StandardScaler()
train_inp = sc.fit_transform(train_inp)
test_inp = sc.transform(test_inp)

In [15]:
del sc
gc.collect()

42

In [16]:
%mkdir ../tmp

In [17]:
dump_pickle(test_inp, '../tmp/test_inp')
del test_inp
gc.collect()

42

In [18]:
train_tar = train_tar.values

## Modeling

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, TensorDataset, random_split

In [20]:
# torch.manual_seed(42)
# torch.backends.cudnn.deterministic = True

In [21]:
%%time
train_inp = torch.from_numpy(train_inp)
train_tar = torch.from_numpy(train_tar)

CPU times: user 29 µs, sys: 6 µs, total: 35 µs
Wall time: 39.3 µs


In [22]:
full_ds = TensorDataset(train_inp, train_tar)
train_sz = 56800
val_sz = len(full_ds) - train_sz
train_ds, val_ds = random_split(full_ds, 
                                [train_sz, val_sz],
                                generator=torch.Generator().manual_seed(42))

In [23]:
batch_size = 512
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

In [24]:
class Net(nn.Module):
    def __init__(self):
        """In the constructor we instantiate two nn.Linear modules and assign them as
        member variables (self).
        """
        super(Net, self).__init__()
        self.linear1 = nn.Linear(20856, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.linear2 = nn.Linear(128, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.linear3 = nn.Linear(128, 128)
        self.bn3 = nn.BatchNorm1d(128)
        self.linear4 = nn.Linear(128, 140)

    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        x = self.linear1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.linear2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.linear3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = self.linear4(x)
        return x

In [25]:
# preds = net(x)
# vpreds = preds - torch.mean(preds)
# vy = y - torch.mean(y)
# loss = torch.sum(vpreds * vy) / \
#        (torch.sqrt(torch.sum(vpreds ** 2)) *
#         torch.sqrt(torch.sum(vy ** 2)))
# loss

In [26]:
def train_model(train_loader, model, optimizer):
    
    model.train()
    sum_corr = 0.0 # sum_loss = 0.0
    total = 0
    
    for i, (x, y) in enumerate(train_loader):
        batch = x.shape[0]
        
        preds = model(x)
        vpreds = preds - torch.mean(preds)
        vy = y - torch.mean(y)
        corr = torch.sum(vpreds * vy) / \
               (torch.sqrt(torch.sum(vpreds ** 2)) *
                torch.sqrt(torch.sum(vy ** 2)))
        loss = -corr
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
        
        total += batch
        sum_corr += batch * corr
    
    train_corr = sum_corr/total 
    return train_corr

In [27]:
def model_eval(model, val_loader):
    model.eval()
    sum_corr = 0.0 # sum_loss = 0.0
    total = 0
    
    for i, (x, y) in enumerate(val_loader):
        batch = x.shape[0]
        
        preds = model(x)
        vpreds = preds - torch.mean(preds)
        vy = y - torch.mean(y)
        corr = torch.sum(vpreds * vy) / \
               (torch.sqrt(torch.sum(vpreds ** 2)) *
                torch.sqrt(torch.sum(vy ** 2)))
        loss = -corr
        
        total += batch
        sum_corr += batch * corr
        
    val_corr = sum_corr/total
    return val_corr

In [28]:
# res = []
# lrs = [0.01, 0.001, 0.0001, 0.00001]
# wds = [0.1, 0.01]
# epochs = 25

# for learning_rate in lrs:
#     for weight_decay in wds:
#         print('------')
#         print(f'learning rate: {learning_rate}')
#         print(f'weight decay: {weight_decay}')
#         print()
        
#         net = Net()
#         optimizer = optim.Adam(net.parameters(),
#                     lr=learning_rate,
#                     weight_decay=weight_decay)
        
#         train_corrs = []
#         val_corrs = []
        
#         for i in range(epochs):
#             print('epoch', i)
#             train_corr = -(train_model(train_loader, net, optimizer))
#             train_corrs.append(float(train_corr.detach().numpy()))
#             print('train corr:', train_corr)
#             val_corr = -(model_eval(net, val_loader))
#             val_corrs.append(float(val_corr.detach().numpy()))
#             print('val corr:', val_corr)
#             print()
        
#         res.append([train_corrs, val_corrs, 
#                     learning_rate, weight_decay])   
        
# del net, train_corrs, val_corrs
# gc.collect()

In [29]:
# figsize(12, 8)
# for i in range(8):
#     train_corrs, val_corrs, learning_rate, weight_decay = res[i]
#     print('------')
#     print(f'learning rate: {learning_rate}')
#     print(f'weight decay: {weight_decay}')
#     print(f'best score on val set: {np.max(val_corrs):.4f}')
#     print()
    
#     plt.subplot(4, 2, i+1)
#     plt.scatter(list(range(epochs)), 
#                 train_corrs, 
#                 label='training corr')
#     plt.plot(list(range(epochs)), 
#              train_corrs)
#     plt.scatter(list(range(epochs)), 
#                 val_corrs, 
#                 label='val corr')
#     plt.plot(list(range(epochs)), 
#              val_corrs)
#     plt.legend(loc='lower right')
#     plt.title(f'lr = {learning_rate} | wd = {weight_decay}')
#     plt.ylim(0.7, 1)
# plt.tight_layout()

# del train_corrs, val_corrs
# gc.collect()

In [30]:
net = Net()
learning_rate = 0.001
weight_decay = 0.01 
optimizer = optim.Adam(net
                       .parameters(),
                       lr=learning_rate,
                       weight_decay=weight_decay)

In [31]:
%%time
epochs = 100
train_corrs = []
val_corrs = []
best_val_corr = 0.0
best_epoch = 0

for epoch in range(epochs):
    # train
    print('epoch', epoch)
    train_corr = train_model(train_loader, net, optimizer)
    train_corr = float(train_corr.detach().numpy())
    train_corrs.append(train_corr)
    print(f'train corr: {train_corr:.4f}')
    
    # val
    val_corr = model_eval(net, val_loader)
    val_corr = float(val_corr.detach().numpy())
    val_corrs.append(val_corr)
    print(f'val corr: {val_corr:.4f}')
    print()
    
    # early stoppage
    if epoch - best_epoch > 5:
        if val_corr < best_val_corr:
            print(f'**Early Stoppage** We are stopping at epoch {epoch}...')
            print(f'Best epoch: epoch {best_epoch}')
            print(f'Best val correlation: {val_corr: .4f}')
            break
            
    # update
    if val_corr > best_val_corr:
        best_val_corr = max([best_val_corr, val_corr])
        best_epoch = epoch
#         print(epoch, best_epoch, best_val_corr)
#         print()

epoch 0
train corr: 0.8043
val corr: 0.8726

epoch 1
train corr: 0.8767
val corr: 0.8734

epoch 2
train corr: 0.8784
val corr: 0.8778

epoch 3
train corr: 0.8798
val corr: 0.8781

epoch 4
train corr: 0.8799
val corr: 0.8755

epoch 5
train corr: 0.8812
val corr: 0.8785

epoch 6
train corr: 0.8811
val corr: 0.8786

epoch 7
train corr: 0.8818
val corr: 0.8798

epoch 8
train corr: 0.8822
val corr: 0.8800

epoch 9
train corr: 0.8828
val corr: 0.8796

epoch 10
train corr: 0.8835
val corr: 0.8806

epoch 11
train corr: 0.8842
val corr: 0.8795

epoch 12
train corr: 0.8846
val corr: 0.8814

epoch 13
train corr: 0.8851
val corr: 0.8812

epoch 14
train corr: 0.8853
val corr: 0.8846

epoch 15
train corr: 0.8855
val corr: 0.8825

epoch 16
train corr: 0.8857
val corr: 0.8818

epoch 17
train corr: 0.8859
val corr: 0.8835

epoch 18
train corr: 0.8863
val corr: 0.8827

epoch 19
train corr: 0.8860
val corr: 0.8813

epoch 20
train corr: 0.8866
val corr: 0.8823

**Early Stoppage** We are stopping at epoch 

In [32]:
# figsize(8, 8)
# plt.scatter(list(range(epochs)), train_corrs, label='training corr')
# plt.plot(list(range(epochs)), train_corrs)
# plt.scatter(list(range(epochs)), val_corrs, label='val corr')
# plt.plot(list(range(epochs)), val_corrs)
# plt.legend(loc='lower right')
# plt.title(f'lr = {learning_rate} | wd = {weight_decay}')
# plt.ylim(0, 1)

In [33]:
# train_inp[:1]

In [34]:
# with torch.no_grad():
#     net.eval()
#     train_tar_preds = net(train_inp[:5]).detach().numpy().flatten()

In [35]:
# np.corrcoef(train_tar_preds, train_tar[:5].detach().numpy().flatten())

In [36]:
del train_inp, train_tar, full_ds
gc.collect()

105

## Prediction

In [37]:
%%time
test_inp = load_pickle('../tmp/test_inp')
test_inp = torch.from_numpy(test_inp)

CPU times: user 1.47 s, sys: 7.7 s, total: 9.17 s
Wall time: 9.15 s


In [38]:
with torch.no_grad():
    net.eval()
    test_tar_preds = net(test_inp).detach().numpy()

In [39]:
del test_inp
gc.collect()

63

## Creating Submission

In [40]:
DATA_DIR = '../input/msci-h5-sparse-transform'
%ls $DATA_DIR -lh

total 7.1G
-rw-r--r-- 1 nobody nogroup  25K Oct 25 08:09 __notebook__.ipynb
-rw-r--r-- 1 nobody nogroup  25K Oct 25 08:09 __output__.json
-rw-r--r-- 1 nobody nogroup 293K Oct 25 08:09 __results__.html
-rw-r--r-- 1 nobody nogroup    0 Oct 25 08:09 custom.css
-rw-r--r-- 1 nobody nogroup 359M Oct 25 08:09 evaluation_ids.parquet
-rw-r--r-- 1 nobody nogroup 3.8M Oct 25 08:09 metadata.parquet
-rw-r--r-- 1 nobody nogroup 108K Oct 25 08:09 metadata_cite_day_2_donor_27678.parquet
-rw-r--r-- 1 nobody nogroup 252M Oct 25 08:09 sample_submission.parquet
-rw-r--r-- 1 nobody nogroup 856K Oct 25 08:09 test_cite_inputs_day_2_donor_27678_idx.npz
-rw-r--r-- 1 nobody nogroup  78M Oct 25 08:09 test_cite_inputs_day_2_donor_27678_val.sparse.npz
-rw-r--r-- 1 nobody nogroup 1.8M Oct 25 08:09 test_cite_inputs_idx.npz
-rw-r--r-- 1 nobody nogroup 488M Oct 25 08:09 test_cite_inputs_val.sparse.npz
-rw-r--r-- 1 nobody nogroup 8.4M Oct 25 08:09 test_multi_inputs_idx.npz
-rw-r--r-- 1 nobody nogroup 1.7G Oct 25 08:09 

In [41]:
test_tar_cols = np.load(f'{DATA_DIR}/train_cite_targets_idx.npz',
                        allow_pickle=True)['columns']
test_tar_idx = np.load(f'{DATA_DIR}/test_cite_inputs_idx.npz',
                       allow_pickle=True)['index']
test_tar_cols.shape, test_tar_idx.shape, test_tar_preds.shape

((140,), (48663,), (48663, 140))

In [42]:
%%time
print('Start Eval...')
eval_ids = pd.read_parquet(f'{DATA_DIR}/evaluation_ids.parquet')
eval_ids.cell_id = eval_ids.cell_id.astype(pd.CategoricalDtype())
eval_ids.gene_id = eval_ids.gene_id.astype(pd.CategoricalDtype())

Start Eval...
CPU times: user 34.5 s, sys: 10.8 s, total: 45.4 s
Wall time: 41 s


In [43]:
%%time
sub = pd.Series(name='target',
                index=pd.MultiIndex.from_frame(eval_ids), 
                dtype=np.float32)
sub

CPU times: user 21.3 s, sys: 7.17 s, total: 28.5 s
Wall time: 28.5 s


row_id    cell_id       gene_id        
0         c2150f55becb  CD86              NaN
1         c2150f55becb  CD274             NaN
2         c2150f55becb  CD270             NaN
3         c2150f55becb  CD155             NaN
4         c2150f55becb  CD112             NaN
                                           ..
65744175  2c53aa67933d  ENSG00000134419   NaN
65744176  2c53aa67933d  ENSG00000186862   NaN
65744177  2c53aa67933d  ENSG00000170959   NaN
65744178  2c53aa67933d  ENSG00000107874   NaN
65744179  2c53aa67933d  ENSG00000166012   NaN
Name: target, Length: 65744180, dtype: float32

In [44]:
cell_id_dict = {cell_id: idx 
                for idx, cell_id in enumerate(test_tar_idx, 0)}
gene_id_dict = {gene_id: idx 
                for idx, gene_id in enumerate(test_tar_cols, 0)}

In [45]:
eid_cid_idx = eval_ids['cell_id']\
              .apply(lambda x: cell_id_dict.get(x, -1))
eid_gid_idx = eval_ids['gene_id']\
              .apply(lambda x: gene_id_dict.get(x, -1))
valid_cite_rows = (eid_cid_idx != -1) & (eid_gid_idx != -1)

In [46]:
%%time
sub.iloc[valid_cite_rows] = test_tar_preds\
                             [eid_cid_idx[valid_cite_rows].to_numpy(),
                              eid_gid_idx[valid_cite_rows].to_numpy()]

CPU times: user 326 ms, sys: 135 ms, total: 462 ms
Wall time: 461 ms


In [47]:
del eval_ids, test_tar_idx, test_tar_cols
del eid_cid_idx, eid_gid_idx, valid_cite_rows
gc.collect()

97

In [48]:
sub = pd.DataFrame(sub).fillna(0).reset_index()
sub.drop(['cell_id', 'gene_id'], axis=1)\
   .to_csv('cite_sub.csv', index=False)

In [49]:
sub.head()

,row_id,cell_id,gene_id,target
0,0,c2150f55becb,CD86,0.000612
1,1,c2150f55becb,CD274,0.000597
2,2,c2150f55becb,CD270,0.001682
3,3,c2150f55becb,CD155,0.007873
4,4,c2150f55becb,CD112,0.008253
